In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
df = data.train_df('small')

In [4]:
df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1.0


In [5]:
df = df.sort_values(['user_id','session_id','timestamp','step']).reset_index()

In [29]:
# find the last clickout rows
last_clickout_idxs = find_last_clickout_indices(df)
clickout_rows = df.loc[last_clickout_idxs, ['user_id','session_id','action_type','impressions','prices']]
# cast the impressions and the prices to lists
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|').apply(lambda x: list(map(int,x)))
clickout_rows['price_list'] = clickout_rows.prices.str.split('|').apply(lambda x: list(map(int,x)))
clickout_rows = clickout_rows.drop('impressions', axis=1)
# order the prices lists
clickout_rows['sorted_price_list'] = clickout_rows.price_list.apply(lambda x: sorted(x))
clickout_rows = clickout_rows.drop('prices', axis=1)

In [18]:
clickout_rows.head()

,user_id,session_id,action_type,impression_list,price_list,sorted_price_list
4,0004IOZI7CKF,0146f7cb014ba,clickout item,"[2632452, 5747484, 110985, 84220, 3752262, 851...","[75, 62, 113, 91, 106, 115, 75, 68, 81, 86, 36...","[36, 62, 65, 66, 68, 69, 72, 75, 75, 81, 82, 8..."
5,0008B0X0HC39,1b703eda68d85,clickout item,[2762974],[494],[494]
16,000ZN9219G41,1086064207e4f,clickout item,"[149015, 1077638, 34819, 4384770, 41629, 55250...","[52, 60, 64, 66, 70, 70, 71, 72, 75, 76, 77, 8...","[52, 60, 64, 66, 70, 70, 71, 72, 75, 76, 77, 8..."
20,001OCXQ9PXET,362dc2be48010,clickout item,"[2450306, 1473305, 6398796, 1161413, 13435, 23...","[67, 84, 65, 79, 103, 74, 76, 98, 73, 98, 62, ...","[55, 56, 62, 65, 67, 73, 74, 74, 74, 75, 76, 7..."
21,001TEVEVUEBE,394f5ad9aa596,clickout item,"[3366614, 2670592, 6625742, 2881004, 5708270, ...","[61, 80, 102, 87, 126, 85, 26, 81, 50]","[26, 50, 61, 80, 81, 85, 87, 102, 126]"


In [30]:
# find the interactions with numeric reference and not last clickouts
reference_rows = df[['user_id','session_id','reference','action_type','index']]
reference_rows = reference_rows[df.reference.str.isnumeric() == True].astype({'reference':'int'})
# skip last clickouts
reference_rows = reference_rows.loc[~reference_rows.index.isin(last_clickout_idxs)]
reference_rows = reference_rows.drop('action_type', axis=1)
#reference_rows['price_pos'] = -1
ref_pos_series = -np.ones(reference_rows.shape[0], dtype=int)

In [31]:
reference_rows.head(30)

,user_id,session_id,reference,index
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777
17,001OCXQ9PXET,362dc2be48010,13857,884525
18,001OCXQ9PXET,362dc2be48010,13857,884526
23,002F746OAPLZ,5958b15f2af92,68760,618712
24,002F746OAPLZ,5958b15f2af92,68760,618713
25,002IXCPKK20W,dcdc75a82f142,2813356,450142
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439
32,003728BB2ALE,b55afcb9b5e96,2516686,618717


In [33]:
# iterate over the sorted reference_rows and clickout_rows
j = 0
clickout_indices = clickout_rows.index.values
ckidx = clickout_indices[j]
next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']
k = 0
for row in tqdm(zip(reference_rows.index, reference_rows.user_id, reference_rows.session_id, 
                    reference_rows.reference)):
    idx = row[0]
    # if the current index is over the last clickout, break
    if idx >= clickout_indices[-1]:
        break
    # find the next clickout index
    while idx > clickout_indices[j]:
        j += 1
        ckidx = clickout_indices[j]
        next_clickout_user_id = clickout_rows.at[ckidx, 'user_id']
        next_clickout_sess_id = clickout_rows.at[ckidx, 'session_id']
        next_clickout_impress = clickout_rows.at[ckidx, 'impression_list']
        next_clickout_prices = clickout_rows.at[ckidx, 'price_list']
        next_clickout_sortedprices = clickout_rows.at[ckidx, 'sorted_price_list']

    # check if row and next_clickout are in the same session
    if row[1] == next_clickout_user_id and row[2] == next_clickout_sess_id:
        try:
            ref_idx = next_clickout_impress.index(row[3])
            ref_price = int(next_clickout_prices[ref_idx])
            ref_pos_series[k] = next_clickout_sortedprices.index(ref_price)
        except:
            pass
    k += 1

In [34]:
reference_rows['price_pos'] = ref_pos_series
reference_rows

,user_id,session_id,reference,index,price_pos
1,0004IOZI7CKF,0146f7cb014ba,3381482,27775,-1
2,0004IOZI7CKF,0146f7cb014ba,2627602,27776,-1
3,0004IOZI7CKF,0146f7cb014ba,7822344,27777,-1
17,001OCXQ9PXET,362dc2be48010,13857,884525,-1
18,001OCXQ9PXET,362dc2be48010,13857,884526,-1
23,002F746OAPLZ,5958b15f2af92,68760,618712,-1
24,002F746OAPLZ,5958b15f2af92,68760,618713,-1
25,002IXCPKK20W,dcdc75a82f142,2813356,450142,-1
27,002O8H8GYKSQ,ab252cd2bafd3,3392402,368439,8
32,003728BB2ALE,b55afcb9b5e96,2516686,618717,-1


In [4]:
from extract_features.rnn.reference_price_position_in_last_clickout import ReferencePricePositionInLastClickout
f = ReferencePricePositionInLastClickout()

In [5]:
f.join_to(df, one_hot=True)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_price_position_in_last_clickout feature read
reference_price_position_in_last_clickout onehot completed


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price_pos_0,price_pos_1,price_pos_2,price_pos_3,price_pos_4,price_pos_5,price_pos_6,price_pos_7,price_pos_8,price_pos_9,price_pos_10,price_pos_11,price_pos_12,price_pos_13,price_pos_14,price_pos_15,price_pos_16,price_pos_17,price_pos_18,price_pos_19,price_pos_20,price_pos_21,price_pos_22,price_pos_23,price_pos_24
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
